## Data Cleaning
___

## Contents

- [Pull Data Into DataFrame](#Pull-Data-Into-DataFrame)
- [Clean 'operator' Target](#Clean-'operator'-Target)
- [Clean 'event_type' Feature](#Clean-'event_type'-Feature)
- [Clean 'event_date' Feature](#Clean-'event_date'-Feature)
- [Airport ID Exploration and Filtering](#Airport-ID-Exploration-and-Filtering)
- [Injury Columns](#Injury-Columns)
- [Cleaning 'probable_cause' column](#Cleaning-'probable_cause'-column)
- [Consolidating 'make' Column](#Consolidating-'make'-Column)
- [Cleaning 'aircraft_damage' Column](#Cleaning-'aircraft_damage'-Column)
- [Cleaning 'model' Column](#Cleaning-'model'-Column)
- [Save Cleaned DataFrame](#Save-Cleaned-DataFrame)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

### Pull Data Into DataFrame
___

In [2]:
airline_df = pd.read_csv('../data/aviation_data_no_commas.csv')

Referenced [Stack Overflow](https://stackoverflow.com/questions/1175208/elegant-python-function-to-convert-camelcase-to-snake-case) for changing camel case to snake case on dataframe columns as well as ChatGPT for regex code.

In [3]:
def camel_to_snake(col):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower()

In [4]:
airline_df.columns = [camel_to_snake(col) for col in airline_df.columns]

In [5]:
airline_df.head(2)

,ntsb_no,event_type,mkey,event_date,city,state,country,report_no,n,has_safety_rec,...,scheduled,purpose_of_flight,f_a_r,air_craft_damage,weather_condition,operator,report_status,rep_gen_flag,docket_url,docket_publish_date
0,DCA24LA051,ACC,193561,2023-12-19T19:30:00Z,St. Louis,Missouri,United States,NaN,N8514F,False,...,SCHD,NaN,121,Substantial,VMC,SOUTHWEST AIRLINES CO,Completed,False,https://data.ntsb.gov/Docket?ProjectID=193561,3/12/2024 18:00
1,DCA24LA034,INC,193459,2023-11-30T08:14:00Z,Kahului,Hawaii,United States,NaN,N494HA,False,...,SCHD,NaN,121,Minor,VMC,Hawaiian Airlines,In work,False,NaN,NaN


In [6]:
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2563 entries, 0 to 2562
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ntsb_no                2563 non-null   object 
 1   event_type             2563 non-null   object 
 2   mkey                   2563 non-null   int64  
 3   event_date             2563 non-null   object 
 4   city                   2558 non-null   object 
 5   state                  2194 non-null   object 
 6   country                2504 non-null   object 
 7   report_no              85 non-null     object 
 8   n                      2559 non-null   object 
 9   has_safety_rec         2563 non-null   bool   
 10  report_type            2301 non-null   object 
 11  original_publish_date  2059 non-null   object 
 12  highest_injury_level   972 non-null    object 
 13  fatal_injury_count     2563 non-null   int64  
 14  serious_injury_count   2563 non-null   int64  
 15  mino

#### Filter out unneeded columns
___

In [7]:
#list of columns worth keeping to streamline cleaning
columns_to_keep = ['event_type',
                   'event_date',
                   'n',
                   'highest_injury_level',
                   'fatal_injury_count',
                   'serious_injury_count',
                   'minor_injury_count',
                   'probable_cause',
                   'latitude',
                   'longitude',
                   'make',
                   'model',
                   'airport_i_d',
                   'air_craft_damage',
                   'operator',
                  ]

airline_filtered_df = airline_df[columns_to_keep].copy()

#### Rename Columns
___

In [8]:
# Change airport_i_d to airport_id and air_craft_damage to aircraft_damage
airline_filtered_df.rename(columns = {'airport_i_d':'airport_id', 'air_craft_damage':'aircraft_damage', 'n':'tail_number'}, inplace = True)

In [9]:
airline_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2563 entries, 0 to 2562
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   event_type            2563 non-null   object 
 1   event_date            2563 non-null   object 
 2   tail_number           2559 non-null   object 
 3   highest_injury_level  972 non-null    object 
 4   fatal_injury_count    2563 non-null   int64  
 5   serious_injury_count  2563 non-null   int64  
 6   minor_injury_count    2563 non-null   int64  
 7   probable_cause        1828 non-null   object 
 8   latitude              2563 non-null   float64
 9   longitude             2563 non-null   float64
 10  make                  2558 non-null   object 
 11  model                 2558 non-null   object 
 12  airport_id            1612 non-null   object 
 13  aircraft_damage       1519 non-null   object 
 14  operator              1776 non-null   object 
dtypes: float64(2), int64(

### Clean 'operator' Target
___

Due to time restraints, we are limiting our target airlines to top ten observations found in the dataset.  If two planes were involved in an accident, both would be recorded in one column cell.  As this rarely happened, we chose to take the first values in all instances where this occured for both operators, damage, and make/model.

In [10]:
#make all lower case
airline_filtered_df['operator'] = airline_filtered_df['operator'].str.lower()

In [11]:
#two operators listed, separated into an operator 2 column
airline_filtered_df[['operator1', 'operator2']] = airline_filtered_df['operator'].str.split(',', n=1, expand=True)

In [12]:
airline_filtered_df[airline_filtered_df['tail_number'] == 'N954U, N441KM']

,event_type,event_date,tail_number,highest_injury_level,fatal_injury_count,serious_injury_count,minor_injury_count,probable_cause,latitude,longitude,make,model,airport_id,aircraft_damage,operator,operator1,operator2
1739,ACC,1994-11-23T00:03:00Z,"N954U, N441KM",Fatal,2,0,0,The Cessna 441 pilot's mistaken belief that hi...,0.0,0.0,"MCDONNELL DOUGLAS, CESSNA","DC-9-82, 441",STL,"Substantial,Destroyed","trans world airlines, inc.,superior aviation, ...",trans world airlines,"inc.,superior aviation, inc."


In [13]:
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('united', na=False), 'operator1'] = 'united'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('delta', na=False), 'operator1'] = 'delta'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('alaska', na=False), 'operator1'] = 'alaska'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('southwest', na=False), 'operator1'] = 'southwest'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('s.w.airlines', na=False), 'operator1'] = 'southwest'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('us', na=False), 'operator1'] = 'us airways'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('frontier', na=False), 'operator1'] = 'frontier'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('continental', na=False), 'operator1'] = 'continental'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('american', na=False), 'operator1'] = 'american'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('eagle', na=False), 'operator1'] = 'american'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('jetblue', na=False), 'operator1'] = 'jetblue'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('blue', na=False), 'operator1'] = 'jetblue'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('spirit', na=False), 'operator1'] = 'spirit'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('hawaiian', na=False), 'operator1'] = 'hawaiian'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('eastern airlines', na=False), 'operator1'] = 'eastern airlines'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('eastern air lines', na=False), 'operator1'] = 'eastern airlines'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('allegiant', na=False), 'operator1'] = 'allegiant'
airline_filtered_df.loc[airline_filtered_df['operator1'].str.contains('sun country', na=False), 'operator1'] = 'sun country'

In [14]:
airline_filtered_df['operator1'].replace(['', ' '], np.nan, inplace = True)
airline_filtered_df['operator1'].dropna(inplace = True)

There are many airlines in this set.  We will narrow it down to top ten airlines within the set.  This includes
* American
* United
* Delta
* Southwest
* US Airways
* Continental
* Hawaiian
* Spirit
* Alaska
* JetBlue
* Frontier
* Eastern
* Allegiant
* Sun Country

In [15]:
#Create list of airlines to keep
airlines = ['american', 'united', 'delta', 'southwest', 'us airways', 'continental', 'hawaiian', 'spirit', 'alaska', 'jetblue', 'frontier', 'eastern', 'allegiant', 'sun country']

In [16]:
#Trim dataset to just keep top airlines - must add copy to prevent slice warnings
airline_filtered_df = airline_filtered_df[airline_filtered_df['operator1'].isin(airlines)].copy()

In [17]:
# 997 observations of top ten airlines
airline_filtered_df.shape

(956, 17)

In [18]:
# Drop second operator columns ane rename operator1 to operator
airline_filtered_df.drop(columns = ['operator','operator2'], inplace = True)
airline_filtered_df.rename(columns = {'operator1':'operator'}, inplace = True)

In [19]:
airline_filtered_df['operator'].value_counts()

operator
american       230
united         197
delta          177
southwest       89
us airways      81
continental     81
alaska          27
jetblue         20
frontier        20
spirit          12
hawaiian        10
allegiant        8
sun country      4
Name: count, dtype: int64

### Clean 'event_type' Feature
___

In [20]:
# Only see five instances of 'OCC' and they all occur at very early rows in the dataset.  None of them look to have any injuries
# listed, so I will replace all values of 'OCC' with 'INC'.
airline_filtered_df['event_type'].value_counts()

event_type
ACC    514
INC    439
OCC      3
Name: count, dtype: int64

In [21]:
airline_filtered_df['event_type'] = airline_filtered_df['event_type'].str.replace('OCC', 'INC')

In [22]:
airline_filtered_df['event_type'].value_counts()

event_type
ACC    514
INC    442
Name: count, dtype: int64

### Clean 'event_date' Feature
___

In [23]:
# Only want the dates in the event date column and then change it to date time
airline_filtered_df['event_date']

0       2023-12-19T19:30:00Z
1       2023-11-30T08:14:00Z
4       2023-09-30T16:00:00Z
5       2023-09-28T09:57:00Z
6       2023-09-25T04:00:00Z
                ...         
2544    1982-05-26T19:55:00Z
2549    1982-04-18T22:24:00Z
2550    1982-04-09T10:52:00Z
2554    1982-02-17T17:45:00Z
2557    1982-02-03T13:02:00Z
Name: event_date, Length: 956, dtype: object

In [24]:
airline_filtered_df['event_date'] = airline_filtered_df['event_date'].str[:10]
airline_filtered_df['event_date'] = pd.to_datetime(airline_filtered_df['event_date'])

In [25]:
airline_filtered_df['event_date']

0      2023-12-19
1      2023-11-30
4      2023-09-30
5      2023-09-28
6      2023-09-25
          ...    
2544   1982-05-26
2549   1982-04-18
2550   1982-04-09
2554   1982-02-17
2557   1982-02-03
Name: event_date, Length: 956, dtype: datetime64[ns]

 ### Airport ID Exploration and Filtering
___

We want to narrow down the dataset to top airport hubs.  I will merge IDs as some of them are in IATA format (LAX) and others are in ICAO format (KLAX).  Because we are just limiting our research to 'on the ground' accidents, we'll drop rows where airport IDs are not listed.  Observations that do not include an aiport ID happen during flight.

In [26]:
airline_filtered_df.dropna(subset = 'airport_id', inplace = True)

Change from IATA ID format to ICAO. We didn't do this for airports with only one one observation since we are going to focus on top airport hubs in the dataset and to save time.


In [27]:
# Make all IDs uppercase
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.upper()

In [28]:
# Drop all rows where there is NONE input instead of no input
airline_filtered_df = airline_filtered_df[airline_filtered_df['airport_id'] != 'NONE']

In [29]:
def airport_code_replace(value):
    if len(value) > 3 and value[0] in ['K', 'P', 'S', 'B', 'L', 'M']:
        return value[1:]
    else:
        return value


In [30]:
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].apply(airport_code_replace)


In [31]:
# The following deal with special cases between IATA and ICAO codes
# in which more than one observation was recording using the different IDs.

airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('RPLB', 'SFS')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('TIST', 'STT')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('D/FW', 'DFW')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('EHAM', 'AMS')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('EGLL', 'LHR')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('YBCG', 'OOL')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('EINN', 'SNN')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('VHHH', 'HKG')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('TAPA', 'ANU')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('NTAA', 'PPT')
airline_filtered_df['airport_id'] = airline_filtered_df['airport_id'].str.replace('CYYC', 'YYC')

In [32]:
airline_filtered_df.shape

(560, 15)

### Injury Columns
___

In [33]:
# Lower characters for highest_injury_level
airline_filtered_df['highest_injury_level'] = airline_filtered_df['highest_injury_level'].str.lower()

In [34]:
# Many of the values here are blank because observations with no injuries were either left blank or set to 'none'.
# Will set to 'none' string.
airline_filtered_df['highest_injury_level'].isnull().sum()

372

In [35]:
airline_filtered_df['highest_injury_level'].fillna('none', inplace = True)
airline_filtered_df['highest_injury_level'].isnull().sum()

0

In [36]:
airline_filtered_df['highest_injury_level'].value_counts()

highest_injury_level
none       372
serious    105
minor       70
fatal       13
Name: count, dtype: int64

In [37]:
# All of the values for injury counts look OK.  No NaNs.
print(airline_filtered_df['fatal_injury_count'].value_counts())
print(airline_filtered_df['serious_injury_count'].value_counts())
print(airline_filtered_df['minor_injury_count'].value_counts())

fatal_injury_count
0      547
1        7
2        2
88       1
27       1
34       1
111      1
Name: count, dtype: int64
serious_injury_count
0     449
1      89
2      10
3       5
5       2
8       1
6       1
9       1
13      1
47      1
Name: count, dtype: int64
minor_injury_count
0      451
1       32
2       17
3       10
5        6
6        6
7        6
4        4
12       3
125      2
18       2
14       2
8        2
25       2
19       2
17       1
36       1
38       1
30       1
88       1
28       1
42       1
27       1
41       1
15       1
20       1
9        1
16       1
Name: count, dtype: int64


### Cleaning 'probable_cause' column
___

We want to use potentially implement natural language processing with this column, but several of them have missing values.  The information that is available looks like it could be useful, so we will fill the empty values with 'NA' as a placeholder.

In [38]:
#Lower strings and fill nulls with 'NA'
airline_filtered_df['probable_cause'] = airline_filtered_df['probable_cause'].str.lower()
airline_filtered_df['probable_cause'].fillna('unknown', inplace = True)

In [39]:
#No NaNs
airline_filtered_df['probable_cause'].isnull().sum()

0

### Consolidating 'make' Column
___

In [40]:
airline_filtered_df['make'] = airline_filtered_df['make'].str.lower()
airline_filtered_df['make'].isnull().sum()

1

In [41]:
#Get rid of NaNs
airline_filtered_df['make'].fillna('unknown', inplace = True)
airline_filtered_df['make'] = airline_filtered_df['make'].str.replace(' ', '_')

Referenced [this](https://saturncloud.io/blog/how-to-split-pandas-dataframe-column-values-in-python/) website when trying to figure out how to split data into two columns.

In [42]:
airline_filtered_df[['make1', 'make2']] = airline_filtered_df['make'].str.split(',', n=1, expand=True)

Splitting by comma to separate aircraft requires its own cleanup.

In [43]:
# Cleanup of company names.  There is no standarization with how these were entered.

def mcdonnell_douglas_replace(value):
    if 'mcdonnell' in value or 'douglas' in value:
        return 'mcdonnell_douglas'
    else:
        return value

def airbus_replace(value):
    if 'airbus' in value:
        return 'airbus'
    else:
        return value

def bombardier_replace(value):
    if 'bombadier' in value or 'bombardier' in value:
        return 'bombardier'
    else:
        return value

def saab_replace(value):
    if 'saab'in value:
        return 'saab'
    else:
        return value


def british_aircraft_replace(value):
    if 'british'in value or 'bac' in value:
        return 'british_aircraft'
    else:
        return value

In [44]:
airline_filtered_df['make1'] = airline_filtered_df['make1'].apply(mcdonnell_douglas_replace)
airline_filtered_df['make1'] = airline_filtered_df['make1'].apply(airbus_replace)
airline_filtered_df['make1'] = airline_filtered_df['make1'].apply(bombardier_replace)
airline_filtered_df['make1'] = airline_filtered_df['make1'].apply(saab_replace)
airline_filtered_df['make1'] = airline_filtered_df['make1'].apply(british_aircraft_replace)


In [45]:
airline_filtered_df['make1'].value_counts()

make1
boeing               318
mcdonnell_douglas    107
airbus                76
embraer               12
aerospatiale           8
saab                   7
british_aircraft       7
lockheed               7
bombardier             5
fokker                 5
de_havilland           3
atr                    1
unknown                1
fairchild              1
short_brothers         1
canadair               1
Name: count, dtype: int64

In [46]:
# Drop second make columns ane rename make1 to make
airline_filtered_df.drop(columns = ['make','make2'], inplace = True)
airline_filtered_df.rename(columns = {'make1':'make'}, inplace = True)

### Cleaning 'aircraft_damage' Column
___

Types of damage categories are None, Minor, Substantial, and Destroyed.  Some values were left blank.  After looking at the data details, we determined they were left blank because there was no damage to the aircraft

In [47]:
#separating damage, will only be using first damage listed
airline_filtered_df[['aircraft_damage1', 'aircraft_damage2']] = airline_filtered_df['aircraft_damage'].str.split(',', n=1, expand=True)

In [48]:
airline_filtered_df['aircraft_damage1'].isnull().sum()

182

In [49]:
#replace NaN with None, after reviewing other columns
airline_filtered_df['aircraft_damage1'] = airline_filtered_df['aircraft_damage1'].replace(np.nan, 'no_damage')

In [50]:
#dropping aircraft_damage2 column (not needed for our models)
airline_filtered_df.drop(columns=['aircraft_damage2', 'aircraft_damage'], inplace = True)


In [51]:
#Get rid of unneeded columns
airline_filtered_df.rename(columns = {'aircraft_damage1':'aircraft_damage'}, inplace = True)

In [52]:
# 23 entries are 'None' which when reading the csv back to a dataframe, make the value a NaN.  Will change 'None' to 'no_damage'.
airline_filtered_df['aircraft_damage'] = airline_filtered_df['aircraft_damage'].replace('None', 'no_damage')

### Cleaning 'model' Column
___

There are many variations over the same model of plan.  I consilidated the models for more interpretability.  Domain knowledge would really help in a case such as this.

In [53]:
airline_filtered_df[['model1', 'model2']] = airline_filtered_df['model'].str.split(',', n=1, expand=True)

In [54]:
airline_filtered_df['model1'] = airline_filtered_df['model1'].str.replace(' ', '-')

In [55]:
#There isn't any useful information for the one row that has no model.  This will be dropped.
airline_filtered_df[airline_filtered_df['model'].isnull()]

,event_type,event_date,tail_number,highest_injury_level,fatal_injury_count,serious_injury_count,minor_injury_count,probable_cause,latitude,longitude,model,airport_id,operator,make,aircraft_damage,model1,model2
716,INC,2008-09-22,",",none,0,0,0,the tower local controller's failure to provid...,40.669101,-89.609878,NaN,PIA,american,unknown,no_damage,NaN,NaN


In [56]:
airline_filtered_df.drop(716, inplace = True)

In [57]:
#Required for specific entries
regex_code = [
    (r'^AA(\d+)$', r'A\1'),          # Replace 'AA320' with 'A320'
    (r'^AA(\d+-\d+)$', r'A\1'),      # Replace 'AA320-232' with 'A320-232'
    (r'^DC-10-10$', r'\bDC-10\b')    # DC-10-10 to DC-10
]

In [58]:
# Functions consolidate plane model names by company

def boeing_replace(value):
    if '707'in value:
        return '707'
    elif '717'in value:
        return '717'
    elif '727'in value:
        return '727'
    elif '737' in value:
        return '737'
    elif '747' in value:
        return '747'
    elif '757' in value:
        return '757'
    elif '767' in value:
        return '767'
    elif '777' in value:
        return '777'
    elif '787' in value:
        return '787'
    else:
        return value

def airbus_replace(value):
    if '310' in value:
        return 'A310'
    elif 'A319'in value:
        return 'A319'
    elif 'A300' in value or 'A-300' in value:
        return 'A300'
    elif 'A320'in value or '320' in value:
        return 'A320'
    elif 'A321'in value:
        return 'A321'
    elif 'A330'in value:
        return 'A330'
    else:
        return value
        
def dc_replace(value):
    if 'DC'in value and '8' in value:
        return 'DC-8'
    elif 'DC'in value and '9' in value:
        return 'DC-9'
    elif 'DC'in value and '10' in value:
        return 'DC-10'
    else:
        return value

def md_replace(value):
    if 'MD'in value and '10' in value:
        return 'MD-10'
    elif 'MD'in value and '11' in value:
        return 'MD-11'
    elif 'MD'in value and '80' in value:
        return 'MD-80'
    elif 'MD'in value and '90' in value:
        return 'MD-90'
    elif 'MD'in value and '88' in value:
        return 'MD-88'
    else:
        return value
        
def l_1011_replace(value):
    if '1011' in value:
        return 'L-1011'
    else:
        return value

def erj_replace(value):
    if '140' in value:
        return 'ERJ-140'
    elif '145' in value:
        return 'ERJ-145'
    elif '145LR' in value:
        return 'ERJ-145LR'
    elif '190' in value:
        return 'ERJ-190'
    else:
        return value

def f28_replace(value):
    if 'MK' in value and '0100' in value:
        return 'FOKKER-100'
    elif 'MK' in value and '4000' in value:
        return 'F28-4000'
    else:
        return value

def dhc_replace(value):
    if 'DHC' in value:
        return 'DASH-8'
    elif 'MK' in value or '4000' in value:
        return 'F28-4000'
    else:
        return value

def ba_replace(value):
    if '146' in value:
        return 'BAE-146'
    else:
        return value

def saab_replace(value):
    if '340' in value or '340B' in value:
        return 'SAAB-340'
    else:
        return value

In [59]:
# Major model cleaning
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(boeing_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(dc_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(airbus_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(l_1011_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(md_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(erj_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(f28_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(dhc_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(ba_replace)
airline_filtered_df['model1'] = airline_filtered_df['model1'].apply(saab_replace)

In [60]:
# Special case in model cleaning
airline_filtered_df.loc[646,'model1'] = 'MD-82'

In [61]:
# Single uses
airline_filtered_df['model1'] = airline_filtered_df['model1'].replace('100', 'FOKKER-100')
airline_filtered_df['model1'] = airline_filtered_df['model1'].replace('SF-34OB', 'SAAB-340')

In [62]:
airline_filtered_df['model1'].value_counts()

model1
737            162
727             49
757             48
DC-8            29
A320            29
767             28
DC-10           21
MD-88           20
A319            14
A300            13
A321            13
777             13
DC-9            13
MD-80           10
MD-82           10
747             10
A330             8
SAAB-340         7
L-1011           7
DASH-8           6
ERJ-145          6
FOKKER-100       3
ATR-42-300       3
MD-83            3
717              3
ERJ-190          3
MD-11            3
BAE-146          3
F28-4000         2
A310             1
SD3-60           1
11-Jan           1
ATP              1
CL44D4           1
707              1
RJ-70            1
EMB-135          1
SA227-AC         1
ATR-42-2000      1
EMB-120ER        1
ATR72-212        1
ERJ-140          1
CL600-2C10       1
ATR72            1
F4-622R          1
CL-600-2C10      1
787              1
MD-90            1
1-11/204/AF      1
Name: count, dtype: int64

In [63]:
# Based on tail number, this is a BAC-111 model plane. Will rename to proper model
airline_filtered_df[airline_filtered_df['model1'] == 'BAC-111']

,event_type,event_date,tail_number,highest_injury_level,fatal_injury_count,serious_injury_count,minor_injury_count,probable_cause,latitude,longitude,model,airport_id,operator,make,aircraft_damage,model1,model2


In [64]:
airline_filtered_df.loc[2429,'model1'] = 'BAC-111'

In [65]:
#dropping aircraft_damage2 column (not needed for our models)
airline_filtered_df.drop(columns=['model2', 'model'], inplace = True)

In [66]:
airline_filtered_df.rename(columns = {'model1':'model'}, inplace = True)

I realize that only have one instances of an airport_id or airline_model will cause warning errors in modeling.  This is something that could be improved  upon given more time.  I acknowledge that this could be a reason why the models won't work well.

In [67]:
print(airline_filtered_df['airport_id'].value_counts())
airline_filtered_df['airport_id'].nunique()


airport_id
ORD    34
DEN    33
ATL    32
LAX    29
EWR    25
       ..
IWA     1
WWR     1
MKE     1
ICN     1
YNG     1
Name: count, Length: 147, dtype: int64


147

In [68]:
print(airline_filtered_df['model'].value_counts())
airline_filtered_df['model'].nunique()


model
737            162
727             49
757             48
DC-8            29
A320            29
767             28
DC-10           21
MD-88           20
A319            14
A300            13
A321            13
777             13
DC-9            13
MD-80           10
MD-82           10
747             10
A330             8
SAAB-340         7
L-1011           7
DASH-8           6
ERJ-145          6
FOKKER-100       3
ATR-42-300       3
MD-83            3
717              3
ERJ-190          3
MD-11            3
BAE-146          3
F28-4000         2
A310             1
SD3-60           1
BAC-111          1
ATP              1
CL44D4           1
707              1
RJ-70            1
EMB-135          1
SA227-AC         1
ATR-42-2000      1
EMB-120ER        1
ATR72-212        1
ERJ-140          1
CL600-2C10       1
ATR72            1
F4-622R          1
CL-600-2C10      1
787              1
MD-90            1
1-11/204/AF      1
Name: count, dtype: int64


49

### Save Cleaned DataFrame
___

In [69]:
#NaN free dataset
airline_filtered_df.isnull().sum()

event_type              0
event_date              0
tail_number             0
highest_injury_level    0
fatal_injury_count      0
serious_injury_count    0
minor_injury_count      0
probable_cause          0
latitude                0
longitude               0
airport_id              0
operator                0
make                    0
aircraft_damage         0
model                   0
dtype: int64

In [70]:
airline_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 559 entries, 1 to 2557
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   event_type            559 non-null    object        
 1   event_date            559 non-null    datetime64[ns]
 2   tail_number           559 non-null    object        
 3   highest_injury_level  559 non-null    object        
 4   fatal_injury_count    559 non-null    int64         
 5   serious_injury_count  559 non-null    int64         
 6   minor_injury_count    559 non-null    int64         
 7   probable_cause        559 non-null    object        
 8   latitude              559 non-null    float64       
 9   longitude             559 non-null    float64       
 10  airport_id            559 non-null    object        
 11  operator              559 non-null    object        
 12  make                  559 non-null    object        
 13  aircraft_damage       55

In [71]:
#airline_filtered_df.to_csv('./data/clean_aviation_data.csv', index = False)